[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_seasonal_index/EMQA_seasonal_index.ipynb)

# EMQA_seasonal_index

Quarterly seasonal index calculation.

**Output:** `seasonal_index_calculation.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
gas = fetch('NG=F', '2023-01-01', '2025-12-31')
quarterly = gas.resample('QE').mean()
quarterly.index = quarterly.index.to_period('Q')

# Manual approach
years = [2023, 2024, 2025]
qtrs = ['Q1','Q2','Q3','Q4']
data = {}
for y in years:
    for qi, q in enumerate(qtrs):
        mask = (gas.index.year == y) & (gas.index.quarter == qi+1)
        vals = gas[mask]
        if len(vals) > 0:
            data.setdefault(q, {})[y] = vals.mean()

print("Quarterly Average Prices (USD/MMBtu):")
print(f"{'':8s} {'2023':>8s} {'2024':>8s} {'2025':>8s} {'Avg':>8s} {'Index':>8s}")
overall_avg = np.mean([np.mean(list(data[q].values())) for q in qtrs])
for q in qtrs:
    vals = [data[q].get(y, np.nan) for y in years]
    avg = np.nanmean(vals)
    idx = avg / overall_avg
    print(f"{q:8s} {vals[0]:>8.2f} {vals[1]:>8.2f} {vals[2]:>8.2f} {avg:>8.2f} {idx:>8.2f}")
print(f"{'Overall':8s} {'':>8s} {'':>8s} {'':>8s} {overall_avg:>8.2f} {'1.00':>8s}")

indices = [np.nanmean(list(data[q].values())) / overall_avg for q in qtrs]
fig, ax = plt.subplots(figsize=(10, 5))
colors_bar = [COLORS['red'] if v > 1 else COLORS['blue'] for v in indices]
ax.bar(qtrs, indices, color=colors_bar, alpha=0.7, width=0.5)
ax.axhline(1.0, color='gray', ls='--', lw=1)
for i, v in enumerate(indices):
    ax.text(i, v + 0.01, f'{v:.2f}', ha='center', fontsize=11, fontweight='bold')
ax.set_title('Quarterly Seasonal Indices: Henry Hub Gas (2023-2025)', fontweight='bold')
ax.set_ylabel('Seasonal Index')
ax.legend(['Index = 1.0'], loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)
fig.tight_layout()
save_fig(fig, 'seasonal_index_calculation.pdf')
plt.show()
